### Respiratoy sound automatic annotation (1D Conv, without spectrograms, no downsampling)
Attempt automatic annotation of sound files in the same format as the provided text files.
Use the sound data directly without creating spectograms

In [1]:
import os
import glob
import re
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import soundfile as sf

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, cross_val_score
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, precision_score, precision_recall_curve, plot_precision_recall_curve
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

import tensorflow as tf
from tensorflow import keras

try:
    os.environ['KAGGLE_DATA_PROXY_TOKEN']
except KeyError:
    dir_out = "./"
    dir_files = "Respiratory_Sound_Database/Respiratory_Sound_Database/"
    dir_annot_csv = "./"
    fname_demo = dir_path + "demographic_info.txt"
else:
    dir_out = "/kaggle/working/"
    dir_files = "/kaggle/input/respiratory-sound-database/Respiratory_Sound_Database/Respiratory_Sound_Database/"
    dir_annot_csv = "/kaggle/input/recording-annotations/"
    fname_demo = "/kaggle/input/respiratory-sound-database/" + "demographic_info.txt"
    
fname_diag = dir_files + "patient_diagnosis.csv"
dir_audio = dir_files + "audio_and_txt_files/"

In [2]:
group_pat_num = "([0-9]{3})"
group_rec_index = "([0-9][a-z][0-9])"
group_chest_loc = "(Tc|Al|Ar|Pl|Pr|Ll|Lr)"
group_acc_modes = "(sc|mc)"
group_equipments = "(AKGC417L|LittC2SE|Litt3200|Meditron)"

regex_info = re.compile("_".join([group_pat_num, group_rec_index, group_chest_loc, group_acc_modes, group_equipments]))

top = os.getcwd()
os.chdir(dir_audio)
fnames = glob.glob("*.wav")

l_wav_rec = []
max_len = 0

for fname in fnames:
    match_info = regex_info.match(fname)
    pat_num = int(match_info.group(1))
    rec_index = match_info.group(2)
    chest_loc = match_info.group(3)
    acc_mode = match_info.group(4)
    equipment = match_info.group(5)
    
    wav_content = sf.read(fname)[0]
    l_wav_rec.append([pat_num, rec_index, chest_loc, wav_content])
    
    if len(wav_content) > max_len:
        max_len = len(wav_content)

os.chdir(top)

# pad all recordings to same length
for i in range(len(l_wav_rec)):
    if len(l_wav_rec[i][3]) < max_len:
        padding = [0] * ( max_len - len(l_wav_rec[i][3]) )
        l_wav_rec[i][3] = np.append(l_wav_rec[i][3], padding)

l_wav_rec.sort(key=lambda subl: (subl[0], subl[1], subl[2]))
wav_cols = ["Patient number", "Recording index", "Chest location", "WAV"]
df_wav_rec = pd.DataFrame(l_wav_rec, columns=wav_cols)

In [3]:
x = np.array([[[[1.0,2.0],[1.0,2.0]],[[2.0,3.0],[2.0,3.0]]]])
y = np.array([[[[2.0,4.0],[2.0,4.0]],[[4.0,6.0],[4.0,6.0]]]])
x = np.array([[[[1.0]],[[2.0]]]])
y = np.array([[[[2.0]],[[2.0]]]])
conv = keras.layers.Conv2D(filters=1,kernel_size=2, strides=1, padding="same", activation="relu")
model = keras.models.Sequential([conv])
model.compile(loss="mse", metrics=["accuracy"], optimizer="sgd")
model.fit(x, y)
conv.get_weights()[0].shape
conv.get_weights()[0]
#x.shape

1/1 [==============================] - 0s 2ms/step - loss: 4.0000 - accuracy: 0.0000e+00


array([[[[-6.3973081e-01]],

        [[-2.9672062e-01]]],


       [[[ 1.6105175e-04]],

        [[ 3.8564199e-01]]]], dtype=float32)

In [4]:
df_annot = pd.read_csv(dir_annot_csv + "rec_annotation.csv")
df_annot

,Patient number,Recording index,Chest location,Cycle number,Cycle start,Cycle end,Crackles,Wheezes
0,178,1b2,Lr,0,0.042,1.280,0,1
1,178,1b2,Lr,1,1.280,2.697,1,1
2,178,1b2,Lr,2,2.697,4.006,1,1
3,178,1b2,Lr,3,4.006,5.506,0,1
4,178,1b2,Lr,4,5.506,7.089,0,1
...,...,...,...,...,...,...,...,...
6893,200,2p4,Al,0,0.175,3.297,0,0
6894,200,2p4,Al,1,3.297,7.805,0,0
6895,200,2p4,Al,2,7.805,11.594,0,0
6896,200,2p4,Al,3,11.594,15.902,0,0


In [9]:
# x1 = np.array([[df_wav_rec.loc[0, "WAV"].astype("float32")]])
x1 = np.array([df_wav_rec.loc[0, "WAV"].astype("float32")]).reshape(-1,1)[np.newaxis,:,:]
x2 = np.array([[df_wav_rec.loc[33, "WAV"].astype("float32")]])
# y = np.array([[x for x in range(len(x1[0][0]))]])
# y = df_annot.loc[0, ]
conv = keras.layers.Conv1D(filters=32,kernel_size=5, strides=2, padding="same", activation="relu")

model = keras.models.Sequential()
# model.add(keras.layers.InputLayer(input_shape=[None, max_len]))
model.add(keras.layers.Conv1D(filters=64,kernel_size=5, strides=2, padding="same", activation="relu"))
model.add(keras.layers.Conv1D(filters=32,kernel_size=5, strides=2, padding="same", activation="relu"))
model.add(keras.layers.Conv1D(filters=32,kernel_size=10, strides=1, padding="same", activation="relu"))

model.compile(loss="mse", metrics=["accuracy"], optimizer="sgd")
# model.fit(x1, y)
# model.fit(x2, y)
t = conv(x1)
t

<tf.Tensor: shape=(1, 441001, 32), dtype=float32, numpy=
array([[[0.        , 0.        , 0.        , ..., 0.        ,
         0.0009102 , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.00045017,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.00045197,
         0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.00046343,
         0.        , 0.        ],
        [0.        , 0.01135096, 0.        , ..., 0.00266782,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.00925649,
         0.        , 0.00560379]]], dtype=float32)>

In [8]:
# np.array([df_wav_rec.loc[0, "WAV"].astype("float32")]).reshape(-1,1)
x1.shape

(1, 882001, 1)